In [1]:
# Import libraries and api keys
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:
# Load CSV file created in WeatherPy
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,25.43,72,8,3.71,PN,1737854597
1,1,jinghong,21.9910,100.7341,14.65,57,0,0.81,CN,1737854598
2,2,taulaga,-11.0553,-171.0883,28.65,66,5,6.29,AS,1737854599
3,3,bethel,41.3712,-73.4140,-3.02,54,0,2.06,US,1737854600
4,4,gaspe,48.8334,-64.4819,-13.88,85,0,1.03,CA,1737854601


In [3]:
# Plot cities on the holoview plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities that fit criteria 

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,port elizabeth,-33.9180,25.5701,23.51,89,0,2.68,ZA,1737854656
79,79,cabo san lucas,22.8909,-109.9124,21.53,77,0,2.57,MX,1737854685
122,122,formosa do rio preto,-11.0483,-45.1931,23.35,68,0,0.11,BR,1737854731
174,174,brisas de zicatela,15.8369,-97.0419,25.10,63,0,1.57,MX,1737854790
351,351,cidreira,-30.1811,-50.2056,23.91,91,0,3.04,BR,1737854988
484,484,punta del este,-34.9667,-54.9500,21.04,80,0,2.68,UY,1737855104
503,503,giyani,-23.3025,30.7187,21.38,83,0,4.41,ZA,1737855165
547,547,hatillo,18.4863,-66.8254,25.99,86,0,1.34,PR,1737855215
550,550,tonj,7.2737,28.6822,22.61,18,0,3.05,SS,1737855218
556,556,boromo,11.7500,-2.9333,24.29,19,0,2.46,BF,1737855224


In [5]:
# Figure out hotel arrangements

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
54,port elizabeth,ZA,-33.9180,25.5701,89,
79,cabo san lucas,MX,22.8909,-109.9124,77,
122,formosa do rio preto,BR,-11.0483,-45.1931,68,
174,brisas de zicatela,MX,15.8369,-97.0419,63,
351,cidreira,BR,-30.1811,-50.2056,91,
484,punta del este,UY,-34.9667,-54.9500,80,
503,giyani,ZA,-23.3025,30.7187,83,
547,hatillo,PR,18.4863,-66.8254,86,
550,tonj,SS,7.2737,28.6822,18,
556,boromo,BF,11.7500,-2.9333,19,


In [6]:
# Find the first hotel located within 10,000 metres 

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

print("Starting hotel search")

# Iterate through hotel_df
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Build parameters
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # JSONify
    name_address = name_address.json()
    
    # Grab the first hotel from results
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Display data
hotel_df

Starting hotel search
200
200
200
200
200
200
200
200
200
200


,City,Country,Lat,Lng,Humidity,Hotel Name
54,port elizabeth,ZA,-33.9180,25.5701,89,Waterford Hotel
79,cabo san lucas,MX,22.8909,-109.9124,77,Comfort Rooms
122,formosa do rio preto,BR,-11.0483,-45.1931,68,Pousada Universo - Formosa do Rio Preto BA
174,brisas de zicatela,MX,15.8369,-97.0419,63,Casa de Olas
351,cidreira,BR,-30.1811,-50.2056,91,Hotel Castelo
484,punta del este,UY,-34.9667,-54.9500,80,Hotel Aquarium
503,giyani,ZA,-23.3025,30.7187,83,No hotel found
547,hatillo,PR,18.4863,-66.8254,86,No hotel found
550,tonj,SS,7.2737,28.6822,18,No hotel found
556,boromo,BF,11.7500,-2.9333,19,Sama Camp


In [7]:
# Plot holoview and show hotels on the map

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

## Insights
1. Geographic Locations:

The map highlights two key points:

Margaret River (Western Australia, near latitude -35, longitude ~115).
Montevideo (South America, near latitude -35, longitude ~-56).
Both cities are located in the Southern Hemisphere and share similar latitudinal positions, potentially allowing for a comparison of their climates or geographical characteristics.

2. Interactive Map Features:

The map offers user-friendly interactive functionalities, such as hover-over details (displaying city names) and zoom/pan controls. A legend at the bottom-right helps users identify cities by their assigned colors, making it easy to interpret.

3. Climatic and Environmental Observations:

Assuming the size of the points corresponds to humidity levels, Montevideo's larger point suggests it experiences higher humidity compared to Margaret River. This visualization provides an opportunity to explore climatic variations or similarities between coastal cities on different continents.